# Object Detection using Tensorflow API
___
Need:
* annotation.csv : csv file with all bounding boxes annotated.
* TFRecord : TFRecord for train and valid dataset (make using annotation.csv)
* Pretrained model: which to use... hmmm
* .config: which to use...

In [6]:
import numpy as np
import pandas as pd
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

In [8]:
# paths
DATA_DIR = 'Z:/input/201809_airbus_ship_detection/'

TRAIN_IMG_DIR = 'Z:/input/201809_airbus_ship_detection/train/'
TEST_IMG_DIR = 'Z:/input/201809_airbus_ship_detection/test/'

## making annotation.csv
___

In [9]:
train = pd.read_csv(DATA_DIR + 'train_ship_bbox.csv')
train.head()

,Unnamed: 0,ImageId,EncodedPixels,num_of_ships,bbox,xmin,ymin,xmax,ymax
0,0,00003e153.jpg,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,1,000155de5.jpg,264661 17 265429 33 266197 33 266965 33 267733...,1.0,"(344, 466, 448, 502)",344.0,466.0,448.0,502.0
2,2,00021ddc3.jpg,101361 1 102128 3 102896 4 103663 6 104430 9 1...,9.0,"(131, 733, 168, 763)",131.0,733.0,168.0,763.0
3,3,00021ddc3.jpg,95225 2 95992 5 96760 7 97527 9 98294 9 99062 ...,9.0,"(123, 756, 136, 768)",123.0,756.0,136.0,768.0
4,4,00021ddc3.jpg,74444 4 75212 4 75980 4 76748 4 77517 3 78285 ...,9.0,"(96, 716, 107, 721)",96.0,716.0,107.0,721.0


In [10]:
# I won't use images with no ships
train.dropna(inplace=True)
train.drop(columns=['Unnamed: 0', 'EncodedPixels', 'num_of_ships', 'bbox'], inplace=True)

train.rename(index=str, columns={'ImageId': 'filename'}, inplace=True)
train.set_index('filename', inplace=True)

train['width'] = 786
train['height'] = 786
train['class'] = 'ships'
train.head()

,xmin,ymin,xmax,ymax,width,height,class
filename,,,,,,,
000155de5.jpg,344.0,466.0,448.0,502.0,786,786,ships
00021ddc3.jpg,131.0,733.0,168.0,763.0,786,786,ships
00021ddc3.jpg,123.0,756.0,136.0,768.0,786,786,ships
00021ddc3.jpg,96.0,716.0,107.0,721.0,786,786,ships
00021ddc3.jpg,195.0,662.0,201.0,666.0,786,786,ships


In [17]:
train.shape

(56030, 7)

In [11]:
train.to_csv(DATA_DIR + 'annotations.csv')

## Generating TFRecord
___

In [16]:
! python create_tf_record.py --csv_input=Z:/input/201809_airbus_ship_detection/annotations.csv --output_path=data/

%d training and %d validation examples. 53228 2802
Successfully created the TFRecords: C:\Users\Gabe\Desktop\wip\201809_airbus_ship_detection\data/train.record
Successfully created the TFRecords: C:\Users\Gabe\Desktop\wip\201809_airbus_ship_detection\data/test.record


## Get the pretrained model
___
While accuracy is important, computation cost is high for me. So I've decided to use faster_rcnn_resnet50_coco.

More models available at [detection zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md)

In [18]:
# What model to download.
MODEL_NAME = 'faster_rcnn_resnet50_coco_2018_01_28'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

### unmark below to download the model
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
tar_file.extractall()

## Get the configuration
___
I'll refer to [faster_rcnn_resnet50_coco.config](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

## Train the model
___

In [ ]:
! python Z:/references/tensorflow-models/research/object_detection/model_main.py \
    --model_dir=Z:/output/201809_airbus_ship_detection/model/ \
    --pipeline_config_path=data/ship_detection.config \
    --alsologtostderr  

In [ ]:
# python Z:/references/tensorflow-models/research/object_detection/model_main.py --model_dir=Z:/output/201809_airbus_ship_detection/model/ --pipeline_config_path=data/ship_detection.config --alsologtostderr  

## Save frozen graph

In [2]:
! python Z:/references/tensorflow-models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path=Z:/output/201809_airbus_ship_detection/model/pipeline.config \
    --trained_checkpoint_prefix=Z:/output/201809_airbus_ship_detection/model/model.ckpt-2772 \
    --output_directory=Z:/output/201809_airbus_ship_detection/model

Parsing Inputs...

=========================Options=============================
-max_depth                  10000
-min_bytes                  0
-min_peak_bytes             0
-min_residual_bytes         0
-min_output_bytes           0
-min_micros                 0
-min_accelerator_micros     0
-min_cpu_micros             0
-min_params                 0
-min_float_ops              0
-min_occurrence             0
-step                       -1
-order_by                   name
-account_type_regexes       _trainable_variables
-start_name_regexes         .*
-trim_name_regexes          .*BatchNorm.*
-show_name_regexes          .*
-hide_name_regexes          
-account_displayed_op_only  true
-select                     params
-output                     stdout:

==================Model Analysis Report======================

Doc:
scope: The nodes in the model graph are organized by their names, which is hierarchical like filesystem.
param: Number of parameters (in the Variable).

Profile:
node

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Please switch to tf.train.get_or_create_global_step
Instructions for updating:
Use `tf.profiler.profile(graph, run_meta, op_log, cmd, options)`. Build `options` with `tf.profiler.ProfileOptionBuilder`. See README.md for details
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
2018-09-10 17:40:33.294318: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX AVX2
2018-09-10 17:40:33.826588: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1405] Found device 0 with properties: 
name: GeForce GTX 1050 major: 6 minor: 1 memoryClockRate(GHz): 1.5185
pciBusID: 0000:01:00.0
totalMemory: 2.00GiB freeMemory: 1.60GiB
2018-09-10 17:40:33.826901: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1484] Adding visible gpu devices: 0
2018-09-10 17:40:34.157858: I tensorflow/core/common_runtim